In [1]:
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os

In [2]:
#run number
yr=2018
run_no = 1
###Segment C.2
from UCModel import model

###Segment C.3
instance = model.create_instance('test.dat')

('Complete:', 'test')


In [3]:
###Segment C.4
opt = SolverFactory("gurobi") ##SolverFactory("cplex")
opt.options["threads"] = 1
H = instance.HorizonHours
K=range(1,H+1)
start = 1 ##1 to 364
end  = 366 ##2 to 366
###Segment C.5
#Space to store results
on=[]
switch=[]

mwh=[]
regup=[]
regdown=[]
nse=[]

for day in range(start,end):
         #load Demand and Reserve time series data
    for i in K:
            instance.HorizonDemand[i] = instance.SimDemand[(day-1)*24+i]
           
    result = opt.solve(instance) ##,tee=True to check number of variables
    instance.solutions.load_from(result)   

In [ ]:
##The following section is for storing and sorting results
    for v in instance.component_objects(Var, active=True):
        varobject = getattr(instance, str(v))
        a=str(v)
       
        if a=='mwh':
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    mwh.append((index[0], day,index[1]+((day-1)*24),varobject[index].value))                                              
      
        if a=='regup':    
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    regup.append((index[0],day,index[1]+((day-1)*24),varobject[index].value))

        if a=='regdown':    
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    regdown.append((index[0],day,index[1]+((day-1)*24),varobject[index].value))
         
        regup_pd=pd.DataFrame(regup,columns=('Generator','Day','Hour','Value'))
        regdown_pd=pd.DataFrame(regdown,columns=('Generator','Day','Hour','Value'))  
        mwh_pd=pd.DataFrame(mwh,columns=('Generator','Day','Hour','Value'))    
    # on_pd=pd.DataFrame(on,columns=('Generator','Day','Hour','Value'))
    # switch_pd=pd.DataFrame(switch,columns=('Generator','Day','Hour','Value'))
    # nse_pd=pd.DataFrame(nse,columns=('Generator','Day','Hour','Value'))

In [ ]:
df_gen_param = pd.read_csv('test_generators.csv',header=0)
gen_name = df_gen_param['name']
gen_maxcap = df_gen_param['maxcap']
gen_heat_rate = df_gen_param['heat_rate']
gen_var_om = df_gen_param['var_om']
gen_fuel_cost = df_gen_param['fuel_cost']
gen_st_cost = df_gen_param['st_cost']

cost= []
price=[]
ls=[]

In [ ]:
for i in range(len(gen_name)):
    cost_e =  gen_heat_rate[i]*gen_fuel_cost[i]+gen_var_om[i]
    cost_fre_u =  gen_heat_rate[i]*gen_fuel_cost[i]+gen_var_om[i]*0.5
    cost_fre_d = gen_heat_rate[i]*gen_fuel_cost[i]+gen_var_om[i]*0.3
    cost.append((gen_name[i],cost_e,cost_fre_u,cost_fre_d))
cost_pd=pd.DataFrame(cost,columns=('Generator','cost_e','cost_fre_u','cost_fre_d'))

for i in range(8760):
    for j in range(len(gen_name)):
        pr_e=(mwh_pd[(i*24+j),'Value']>0).astype(int)*1*cost[j,'cost_e']
        pr_fre_u=(mwh_pd[(i*24+j),'Value']>0).astype(int)*1*cost[j,'cost_fre_u']
        pr_fre_d=(mwh_pd[(i*24+j),'Value']>0).astype(int)*1*cost[j,'cost_fre_d']
        ls.append((pr_e,pr_fre_u,pr_fre_d))
    price.append(max(ls))
    
print(price)